In [ ]:
import os
import dask
import fsspec
import netrc
from urllib.parse import urlparse
from urllib.error import HTTPError, URLError
import rioxarray
import xarray as xr

URS_URL = "https://urs.earthdata.nasa.gov"
import base64
from urllib.request import (
    urlopen,
    Request,
    build_opener,
    HTTPCookieProcessor,
)


from tqdm import tqdm

In [ ]:
with open("./srtm_granules.txt") as f:
    granules = f.read().split("\n")

In [ ]:
granules[:10]

In [ ]:
def get_username():
    username = ""

    # For Python 2/3 compatibility:
    try:
        do_input = raw_input  # noqa
    except NameError:
        do_input = input

    while not username:
        try:
            username = do_input("Earthdata username: ")
        except KeyboardInterrupt:
            quit()
    return username


def get_credentials(url):
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    errprefix = ""
    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(
            urlparse(URS_URL).hostname
        )
        errprefix = "netrc error: "
    except Exception as e:
        if not ("No such file" in str(e)):
            print("netrc error: {0}".format(str(e)))
        username = None
        password = None

    while not credentials:
        if not username:
            username = get_username()
            password = get_password()
        credentials = "{0}:{1}".format(username, password)
        credentials = base64.b64encode(credentials.encode("ascii")).decode(
            "ascii"
        )

        if url:
            try:
                req = Request(url)
                req.add_header("Authorization", "Basic {0}".format(credentials))
                opener = build_opener(HTTPCookieProcessor())
                opener.open(req)
            except HTTPError:
                print(errprefix + "Incorrect username or password")
                errprefix = ""
                credentials = None
                username = None
                password = None

    return credentials

In [ ]:
def cmr_download(urls, cache_location, credentials=None):
    """Download files from list of urls."""
    out = []
    if not urls:
        return out

    for index, url in enumerate(urls, start=1):
        if not credentials and urlparse(url).scheme == "https":
            credentials = get_credentials(url)

        filename = url.split("/")[-1]
        target_url = os.path.join(cache_location, filename)

        # skip existing
        try:
            fsspec.open(target_url).open()
            out.append(target_url)
            continue
        except FileNotFoundError:
            pass

        try:
            req = Request(url)
            if credentials:
                req.add_header("Authorization", "Basic {0}".format(credentials))
            opener = build_opener(HTTPCookieProcessor())

            with fsspec.open(target_url, mode="wb") as target:
                target.write(opener.open(req).read())

            out.append(target_url)

        except HTTPError as e:
            print("HTTPError {0}, {1}".format(e.code, e.reason), filename)
        except URLError as e:
            print("URLError: {0}".format(e.reason), filename)
        except IOError:
            print("IOError: {0}".format(e.reason), filename)
        except KeyboardInterrupt:
            quit()
        except:
            print("unknown error", filename)
    return out

In [ ]:
@dask.delayed
def process_granule(url, temp_dir="./temp"):
    target = urlparse(url).path.split("/")[-1].replace("hgt.zip", "tif")
    target_url = f"gs://carbonplan-climatetrace/inputs/srtm/{target}"

    try:
        fsspec.open(target_url).open()
        return target_url
    except FileNotFoundError:
        pass

    # download the granule
    try:
        local_path = cmr_download([url], f"{temp_dir}")[0]
    except Exception as e:
        print(url, e)
        return ("failed to download", target_url)

    # convert to geotiff
    try:
        da = xr.open_rasterio(local_path)
    except Exception as e:
        print(url, e)
        os.remove(local_path)
        return ("failed to convert", target_url)
    cog = f"{temp_dir}/{target}"
    da.rio.to_raster(cog, windowed=True)

    # write to object store
    with fsspec.open(target_url, mode="wb") as fdst:
        with open(cog, mode="rb") as fsrc:
            fdst.write(fsrc.read())

    # cleanup
    os.remove(local_path)
    os.remove(cog)

    return ("passed", target_url)

In [ ]:
results = []
for url in tqdm(granules):
    results.append(process_granule(url))

In [ ]:
results_c = dask.compute(results, retries=3)

In [ ]:
print("here")